In [1]:
import pandas as pd
from src.seq2seq.dataset import SeqDataset, pad_batch
from src.seq2seq.metrics import compute_metrics
from torch.utils.data import DataLoader

DATA_PATH = 'data/ArchiveII/ArchiveII_seq_sim_pp_motifs/train_0.csv'
df = pd.read_csv(DATA_PATH)

display(df.sample(2))
data = SeqDataset(dataset_path=DATA_PATH)
# print(data[0])
for k, i in data[0].items():
    print(f'key: {k},   type:{type(i)}')

,Unnamed: 0,id,fam,sequence,len,structure,base_pairs,pseudo_probe,motifs,stem
772,4735,tRNA_tdbR00000160-Escherichia_coli-562-Ile-AU,tRNA,GGCCCCUUAGCUCAGUGGUUAGAGCAGGCGACUCAUAAUCGCUUGG...,76,(((((((..((((........))))((((((.......)))))).....,"[[1, 72], [2, 71], [3, 70], [4, 69], [5, 68], ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, ...",SSSSSSSMMSSSSHHHHHHHHSSSSSSSSSSHHHHHHHSSSSSSMM...,"[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, ..."
1414,8605,tRNA_tdbR00000148-Ovis_aries-9940-His-QUG,tRNA,GGCCGUGAUCGUAUAGUGGUUAGUACUCUGCGUUGUGGCCGCAGCA...,76,.(((((((..((((........)))).(((((.......))))).....,"[[2, 72], [3, 71], [4, 70], [5, 69], [6, 68], ...","[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, ...",ESSSSSSSMMSSSSHHHHHHHHSSSSMSSSSSHHHHHHHSSSSSMM...,"[0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, ..."


key: id,   type:<class 'str'>
key: length,   type:<class 'int'>
key: sequence,   type:<class 'str'>
key: embedding,   type:<class 'torch.Tensor'>
key: pseudo_probing,   type:<class 'torch.Tensor'>
key: stem,   type:<class 'torch.Tensor'>
key: motif_emb,   type:<class 'torch.Tensor'>


In [2]:
from functools import partial
pad_batch_with_fixed_length = partial(pad_batch, fixed_length=128)
batch_size = 1
loader = DataLoader(
    SeqDataset(DATA_PATH),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    collate_fn=pad_batch_with_fixed_length,
)
load = iter(loader)
d = next(load)
d2 = next(load)

In [3]:
compute_metrics(d['motif_emb'],d2['motif_emb'], d2['mask'], binary=True)

UnboundLocalError: local variable 'true_idx' referenced before assignment

In [ ]:
print(d2['mask'].shape)
print(d2['embedding'].shape)

torch.Size([1, 128])
torch.Size([1, 4, 128])


In [ ]:
d['pseudo_probing'] - d2['pseudo_probing']


tensor([[[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0., -1., -1., -1.,  0.,
           0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0., -1., -1., -1., -1., -1.,
           0.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0., -1., -1., -1., -1.,
          -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  1.,  0.,
           0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.,  0., -1.,  0.,
           0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
           0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,
           1.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
           1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
           0.,  0.]]])

In [24]:
import torch as tr
x = tr.Tensor([[[ 1.,  1.,  1.], ], 
               [[ 1.,  1.,  1.], ], 
               ])

y = tr.Tensor([[[ 1.,  0.,  0.], ], 
               [[ 0.,  0.,  0.],], 
               ])
mask = tr.zeros(2, 3, dtype= tr.bool)
mask[:,:] = True

print(x.shape)
print(y.shape)
print(mask.shape)
mask

torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 3])


tensor([[True, True, True],
        [True, True, True]])

In [25]:
TP = (x.permute(0,2,1)[mask] == y.permute(0,2,1)[mask]).sum().item()   
total = mask.sum().item()
accuracy = TP / total if total > 0 else 0.0
print(f'TP: {TP}, total: {total}, acc: {accuracy}')

compute_metrics(x,y,mask, binary=True)

TP: 1, total: 6, acc: 0.16666666666666666


{'F1': 0.2857142857142857,
 'Accuracy': 0.16666666666666666,
 'Accuracy_seq': 0.0}

In [ ]:
x.argmax(dim=1)

tensor([[0],
        [0]])

In [ ]:
y.argmax(dim=1)

tensor([[0],
        [0]])